In [482]:
#Ethereum trading algo using SVM(support vector machines), SMAs(simple moving averages),Alpaca api data
# Initial imports
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
import os
import requests


In [627]:
# Load .env environment variables

import datetime as dt
import pytz

_timeNow = dt.datetime.now(pytz.timezone('US/Eastern'))
_1YearsAgo = _timeNow - dt.timedelta(days=360)
_2YearsAgo = _timeNow - dt.timedelta(days=730)
_3YearsAgo = _timeNow - dt.timedelta(days=900)
ALPACA_API_BASE_URL = "https://paper-api.alpaca.markets/v2/assets?asset_class=crypto"
# Create a connection to the API 
api = tradeapi.REST("PKHBLCB7QI73L4912M0G", "Aq2JSZktyKcSKaFWq1VxIPn3EOFJB0ICvB7fdmpE", ALPACA_API_BASE_URL, api_version="v2")

In [628]:
# Set the ticker symbol and the number of coins to buy
ticker = "ETHUSD"
number_of_coins = 1

# Make API call
ETH_prices = api.get_crypto_bars(ticker, '1day',
                         start=_3YearsAgo.isoformat(),
                         end=None,
                         limit=None).df

In [629]:
#Filter data by exchange
ETH_prices = ETH_prices[ETH_prices.exchange == 'CBSE']

In [630]:
ETH_prices

,exchange,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,,
2020-05-11 05:00:00+00:00,CBSE,187.22,193.77,176.43,185.93,236384.241453,43748,184.951296
2020-05-12 05:00:00+00:00,CBSE,189.93,191.68,189.49,190.75,18590.092927,5754,190.729780
2020-05-13 05:00:00+00:00,CBSE,199.92,201.27,196.78,197.72,18896.732096,6356,198.757819
2020-05-14 05:00:00+00:00,CBSE,197.71,206.25,195.70,203.40,150803.416403,34301,201.769335
2020-05-15 05:00:00+00:00,CBSE,194.66,201.83,193.24,201.75,29306.112870,7572,197.863064
...,...,...,...,...,...,...,...,...
2022-10-23 05:00:00+00:00,CBSE,1309.95,1370.90,1299.47,1347.90,215025.257380,314968,1340.685134
2022-10-24 05:00:00+00:00,CBSE,1347.89,1359.86,1324.04,1346.60,328802.168167,478197,1341.928261
2022-10-25 05:00:00+00:00,CBSE,1346.60,1525.38,1339.39,1480.84,804882.831580,737655,1440.692615


In [631]:
# Slice to just the `close` column
ETH_signals_df = ETH_prices.loc[:,["close"]]

In [632]:
# Set the short window and long windows
fast_window = 15
slow_window = 50

In [633]:
# Generate the short and long moving averages (50 and 100 days, respectively)
ETH_signals_df['SMA_fast'] = ETH_signals_df['close'].rolling(window=fast_window).mean()
ETH_signals_df['SMA_slow'] = ETH_signals_df['close'].rolling(window=slow_window).mean()

# Prepopulate the `Signal` for trading
ETH_signals_df['Signal'] = 0.0
ETH_signals_df = ETH_signals_df.dropna()


In [634]:
# Generate the trading signal 0 or 1,
# where 1 is when the fast-window (SMA50) is greater than the slow (SMA 100)
# and 0 otherwise

ETH_signals_df["Signal"][fast_window:] = np.where(
    ETH_signals_df["SMA_fast"][fast_window:] > ETH_signals_df["SMA_slow"][fast_window:], 1.0, 0.0)

In [635]:
ETH_signals_df

,close,SMA_fast,SMA_slow,Signal
timestamp,,,,
2020-06-29 05:00:00+00:00,226.33,231.850000,224.6568,0.0
2020-06-30 05:00:00+00:00,225.67,231.496000,225.4516,0.0
2020-07-01 05:00:00+00:00,231.03,231.356000,226.2572,0.0
2020-07-02 05:00:00+00:00,226.48,230.874000,226.8324,0.0
2020-07-03 05:00:00+00:00,226.10,230.740667,227.2864,0.0
...,...,...,...,...
2022-10-23 05:00:00+00:00,1347.90,1304.550667,1400.7362,0.0
2022-10-24 05:00:00+00:00,1346.60,1306.006667,1396.1220,0.0
2022-10-25 05:00:00+00:00,1480.84,1319.584667,1392.9224,0.0


In [636]:
##### Calculate the points in time when the Signal value changes
# Identify trade entry (1) and exit (-1) points
ETH_signals_df['Entry/Exit'] = ETH_signals_df['Signal'].diff()

# Review the DataFrame
ETH_signals_df.loc["2021-02-02":"2022-06-20",:]

,close,SMA_fast,SMA_slow,Signal,Entry/Exit
timestamp,,,,,
2021-02-02 06:00:00+00:00,1545.34,1328.331333,1023.7650,1.0,0.0
2021-02-03 06:00:00+00:00,1657.11,1347.602667,1045.2234,1.0,0.0
2021-02-04 06:00:00+00:00,1597.55,1367.044667,1064.2106,1.0,0.0
2021-02-05 06:00:00+00:00,1662.98,1403.868667,1084.6028,1.0,0.0
2021-02-06 06:00:00+00:00,1616.61,1429.402667,1103.8704,1.0,0.0
...,...,...,...,...,...
2022-06-16 05:00:00+00:00,1084.99,1561.312000,2026.7366,0.0,0.0
2022-06-17 05:00:00+00:00,1079.22,1512.249333,1989.9510,0.0,0.0
2022-06-18 05:00:00+00:00,966.60,1458.893333,1952.8472,0.0,0.0


In [637]:
# Visualize exit position relative to close price
exit = ETH_signals_df[ETH_signals_df['Entry/Exit'] == -1.0]['close'].hvplot.scatter(
    color='yellow',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

In [638]:
# Visualize entry position relative to close price
entry = ETH_signals_df[ETH_signals_df['Entry/Exit'] == 1.0]['close'].hvplot.scatter(
    color='purple',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

In [639]:
# Visualize close price for the investment
security_close =ETH_signals_df[['close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

In [640]:
# Visualize moving averages
moving_avgs = ETH_signals_df[['SMA_fast', 'SMA_slow']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

In [641]:
moving_avgs

:NdOverlay   [Variable]
   :Curve   [timestamp]   (value)

In [642]:
# Create the overlay plot
entry_exit_plot = entry * exit

# Show the plot with a title
entry_exit_plot.opts(
    title="ETH - SMA_fast, SMA_slow, Entry and Exit Points")

:Overlay
   .Scatter.Close.I  :Scatter   [timestamp]   (close)
   .Scatter.Close.II :Scatter   [timestamp]   (close)

In [643]:
total_plot=entry_exit_plot*moving_avgs
total_plot

:Overlay
   .Scatter.Close.I  :Scatter   [timestamp]   (close)
   .Scatter.Close.II :Scatter   [timestamp]   (close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [timestamp]   (value)

In [644]:
# Set initial capital
initial_capital = float(100000)

# Set the share size
share_size = 50

In [645]:
# Buy when the dual moving average crossover Signal equals 1
# Otherwise, `Position` should be zero (sell)
ETH_signals_df['Position'] = share_size * ETH_signals_df['Signal']

In [646]:
# Determine the points in time where a position is bought or sold
ETH_signals_df['Entry/Exit Position'] = ETH_signals_df['Position'].diff()

In [647]:
# Multiply the close price by the number of shares held, or the Position
ETH_signals_df['Portfolio Holdings'] = ETH_signals_df['close'] * ETH_signals_df['Position']

# Subtract the amount of either the cost or proceeds of the trade from the initial capital invested
ETH_signals_df['Portfolio Cash'] = (initial_capital - (ETH_signals_df["close"] * ETH_signals_df["Entry/Exit Position"]).cumsum())

In [648]:
# Calculate the total portfolio value by adding the portfolio cash to the portfolio holdings (or investments)
ETH_signals_df['Portfolio Total'] = ETH_signals_df['Portfolio Cash'] + ETH_signals_df['Portfolio Holdings']

# Conclusion:  each point at a time, you can see what you have is 
# 1) if you are holding stock. say 500 share, what's holding value of 500 share at that moment + (initial capital + net of cost and proceed)
# in each row, you have  holding = position * close + initial investment - cost of previous buys and + proceed of previous sales

In [649]:
# Calculate the portfolio daily returns
ETH_signals_df["Portfolio Daily Returns"] = ETH_signals_df["Portfolio Total"].pct_change()

# Calculate the cumulative returns
ETH_signals_df["Portfolio Cumulative Returns"] = (
    1 + ETH_signals_df["Portfolio Daily Returns"]
).cumprod() - 1

# Print the DataFrame
ETH_signals_df

,close,SMA_fast,SMA_slow,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
timestamp,,,,,,,,,,,,
2020-06-29 05:00:00+00:00,226.33,231.850000,224.6568,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN
2020-06-30 05:00:00+00:00,225.67,231.496000,225.4516,0.0,0.0,0.0,0.0,0.0,100000.0,100000.0,NaN,NaN
2020-07-01 05:00:00+00:00,231.03,231.356000,226.2572,0.0,0.0,0.0,0.0,0.0,100000.0,100000.0,0.0,0.000000
2020-07-02 05:00:00+00:00,226.48,230.874000,226.8324,0.0,0.0,0.0,0.0,0.0,100000.0,100000.0,0.0,0.000000
2020-07-03 05:00:00+00:00,226.10,230.740667,227.2864,0.0,0.0,0.0,0.0,0.0,100000.0,100000.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-23 05:00:00+00:00,1347.90,1304.550667,1400.7362,0.0,0.0,0.0,0.0,0.0,224773.5,224773.5,0.0,1.247735
2022-10-24 05:00:00+00:00,1346.60,1306.006667,1396.1220,0.0,0.0,0.0,0.0,0.0,224773.5,224773.5,0.0,1.247735
2022-10-25 05:00:00+00:00,1480.84,1319.584667,1392.9224,0.0,0.0,0.0,0.0,0.0,224773.5,224773.5,0.0,1.247735


In [650]:
# Create the list of the metric names
metrics = [
    'Annualized Return',
    'Cumulative Returns',
    'Annual Volatility',
    'Sharpe Ratio',
    'Sortino Ratio'
]

# Create a list that holds the column name
columns = ['Backtest']

# Initialize the DataFrame with index set to evaluation metrics and columns 
portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)

# Review the DataFrame
portfolio_evaluation_df

,Backtest
Annualized Return,NaN
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [651]:
# Calculate the Annualized return metric
portfolio_evaluation_df.loc['Annualized Return'] = (
    ETH_signals_df['Portfolio Daily Returns'].mean() * 252)

In [652]:
# Calculate the Cumulative returns metric
portfolio_evaluation_df.loc['Cumulative Returns'] = ETH_signals_df['Portfolio Cumulative Returns'][-1]

In [653]:
# Calculate the Annual volatility metric
portfolio_evaluation_df.loc['Annual Volatility'] = (
    ETH_signals_df['Portfolio Daily Returns'].std() * np.sqrt(252))

In [654]:
# Calculate the Sharpe ratio
portfolio_evaluation_df.loc['Sharpe Ratio'] = (
    ETH_signals_df['Portfolio Daily Returns'].mean() * 252) / (
    ETH_signals_df['Portfolio Daily Returns'].std() * np.sqrt(252))

In [655]:
# Calculate the Sortino ratio
# Start by calculating the downside return values

# Create a DataFrame that contains the Portfolio Daily Returns column
sortino_ratio_df = ETH_signals_df[['Portfolio Daily Returns']].copy()

# Create a column to hold downside return values
sortino_ratio_df.loc[:,'Downside Returns'] = 0

# Find Portfolio Daily Returns values less than 0, 
# square those values, and add them to the Downside Returns column
sortino_ratio_df.loc[sortino_ratio_df['Portfolio Daily Returns'] < 0, 
                     'Downside Returns'] = sortino_ratio_df['Portfolio Daily Returns']**2

# Calculate the annualized return value
annualized_return = sortino_ratio_df['Portfolio Daily Returns'].mean() * 252

# Calculate the annualized downside standard deviation value
downside_standard_deviation = np.sqrt(sortino_ratio_df['Downside Returns'].mean()) * np.sqrt(252)

# Divide the annualized return value by the downside standard deviation value
sortino_ratio = annualized_return/downside_standard_deviation

# Add the Sortino ratio to the evaluation DataFrame
portfolio_evaluation_df.loc['Sortino Ratio'] = sortino_ratio

In [656]:
# Review the portfolio evaluation DataFrame
portfolio_evaluation_df

,Backtest
Annualized Return,0.292566
Cumulative Returns,1.247735
Annual Volatility,0.322373
Sharpe Ratio,0.907539
Sortino Ratio,1.330369


In [657]:
# Initialize the trade evaluation DataFrame
trade_evaluation_df = pd.DataFrame(
    columns=[
        'Crypto', 
        'Entry Date', 
        'Exit Date', 
        'Coins',
        'Entry Coin Price',
        'Exit Coin Price',
        'Entry Portfolio Holding', 
        'Exit Portfolio Holding', 
        'Profit/Loss'])

In [658]:
# Initialize the iterative variables
entry_date = ""
exit_date = ""
entry_portfolio_holding = 0.0
exit_portfolio_holding = 0.0
share_size = 0
entry_coin_price = 0.0
exit_coin_price = 0.0

# Loop through the signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit,
# Then append the record to the trade evaluation DataFrame
for index, row in ETH_signals_df.iterrows():
    if row['Entry/Exit'] == 1:
        entry_date = index
        entry_portfolio_holding = abs(row['Portfolio Holdings'])
        share_size = row['Entry/Exit Position']
        entry_coin_price = row['close']

    elif row['Entry/Exit'] == -1:
        exit_date = index
        exit_portfolio_holding = abs(row['close'] * row['Entry/Exit Position'])
        exit_coin_price = row['close']
        profit_loss =  exit_portfolio_holding - entry_portfolio_holding
        trade_evaluation_df = trade_evaluation_df.append(
            {
                'Crypto': 'ETH',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Coins': share_size,
                'Entry Coin Price': entry_coin_price,
                'Exit Coin Price': exit_coin_price,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Review the DataFrame
trade_evaluation_df


C:\Users\cstan\AppData\Local\Temp\ipykernel_23004\2749651264.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trade_evaluation_df = trade_evaluation_df.append(
C:\Users\cstan\AppData\Local\Temp\ipykernel_23004\2749651264.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trade_evaluation_df = trade_evaluation_df.append(
C:\Users\cstan\AppData\Local\Temp\ipykernel_23004\2749651264.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trade_evaluation_df = trade_evaluation_df.append(
C:\Users\cstan\AppData\Local\Temp\ipykernel_23004\2749651264.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trade_evaluation_df = trade_evaluation

,Crypto,Entry Date,Exit Date,Coins,Entry Coin Price,Exit Coin Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,ETH,2020-07-14 05:00:00+00:00,2020-09-13 05:00:00+00:00,50.0,239.89,364.56,11994.5,18228.0,6233.5
1,ETH,2020-10-20 05:00:00+00:00,2021-05-29 05:00:00+00:00,50.0,381.51,2299.17,19075.5,114958.5,95883.0
2,ETH,2021-07-31 05:00:00+00:00,2021-09-23 05:00:00+00:00,50.0,2621.51,3081.03,131075.5,154051.5,22976.0
3,ETH,2021-10-12 05:00:00+00:00,2021-11-29 06:00:00+00:00,50.0,3508.25,4389.78,175412.5,219489.0,44076.5
4,ETH,2021-12-01 06:00:00+00:00,2021-12-03 06:00:00+00:00,50.0,4528.57,3783.89,226428.5,189194.5,-37234.0
5,ETH,2022-03-25 05:00:00+00:00,2022-04-25 05:00:00+00:00,50.0,3113.85,3006.36,155692.5,150318.0,-5374.5
6,ETH,2022-07-22 05:00:00+00:00,2022-08-31 05:00:00+00:00,50.0,1587.32,1551.58,79366.0,77579.0,-1787.0


In [659]:
#Machine Learning with SVM

In [660]:
#Actual Returns
ETH_signals_df["Actual Returns"] = ETH_signals_df["close"].pct_change()

In [661]:
#Train Test begin
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report

In [662]:
# Assign a copy of the sma_fast and sma_slow columns to a features DataFrame called X
X = ETH_signals_df[['SMA_fast', 'SMA_slow']].shift().dropna()

# Review the DataFrame
X

,SMA_fast,SMA_slow
timestamp,,
2020-06-30 05:00:00+00:00,231.850000,224.6568
2020-07-01 05:00:00+00:00,231.496000,225.4516
2020-07-02 05:00:00+00:00,231.356000,226.2572
2020-07-03 05:00:00+00:00,230.874000,226.8324
2020-07-04 05:00:00+00:00,230.740667,227.2864
...,...,...
2022-10-23 05:00:00+00:00,1302.236667,1404.9070
2022-10-24 05:00:00+00:00,1304.550667,1400.7362
2022-10-25 05:00:00+00:00,1306.006667,1396.1220


In [663]:
# Create the target set selecting the Signal column and assiging it to y
y = ETH_signals_df['Signal']

# Review the value counts
y.value_counts()

1.0    457
0.0    394
Name: Signal, dtype: int64

In [664]:
# Select the start of the training period
training_begin = X.index.min()

# Display the training begin date
print(training_begin)

2020-06-30 05:00:00+00:00


In [665]:
# Select the ending period for the training data with an offset of 1 months
training_end = X.index.min() + DateOffset(months=16)

# Display the training end date
print(training_end)

2021-10-30 05:00:00+00:00


In [666]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Review the X_train DataFrame
display(X_train.head())
display(X_train.tail())

,SMA_fast,SMA_slow
timestamp,,
2020-06-30 05:00:00+00:00,231.850000,224.6568
2020-07-01 05:00:00+00:00,231.496000,225.4516
2020-07-02 05:00:00+00:00,231.356000,226.2572
2020-07-03 05:00:00+00:00,230.874000,226.8324
2020-07-04 05:00:00+00:00,230.740667,227.2864


,SMA_fast,SMA_slow
timestamp,,
2021-10-26 05:00:00+00:00,3903.307333,3484.2914
2021-10-27 05:00:00+00:00,3955.374667,3491.5206
2021-10-28 05:00:00+00:00,3987.948000,3502.7184
2021-10-29 05:00:00+00:00,4034.708000,3519.8318
2021-10-30 05:00:00+00:00,4068.987333,3537.5286


In [667]:
display(y_train.head())
display(y_train.tail())
display(y_train.describe())
y_train.value_counts()

timestamp
2020-06-30 05:00:00+00:00    0.0
2020-07-01 05:00:00+00:00    0.0
2020-07-02 05:00:00+00:00    0.0
2020-07-03 05:00:00+00:00    0.0
2020-07-04 05:00:00+00:00    0.0
Name: Signal, dtype: float64

timestamp
2021-10-26 05:00:00+00:00    1.0
2021-10-27 05:00:00+00:00    1.0
2021-10-28 05:00:00+00:00    1.0
2021-10-29 05:00:00+00:00    1.0
2021-10-30 05:00:00+00:00    1.0
Name: Signal, dtype: float64

count    488.000000
mean       0.727459
std        0.445724
min        0.000000
25%        0.000000
50%        1.000000
75%        1.000000
max        1.000000
Name: Signal, dtype: float64

1.0    355
0.0    133
Name: Signal, dtype: int64

In [668]:
X_train.describe()

,SMA_fast,SMA_slow
count,488.000000,488.000000
mean,1631.210448,1511.627727
std,1163.605991,1099.283115
min,230.551333,224.656800
25%,406.834833,386.303700
50%,1686.400667,1492.277300
75%,2551.436833,2479.455750
max,4068.987333,3537.528600


In [669]:
# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end+DateOffset(hours=1):]
y_test = y.loc[training_end+DateOffset(hours=1):]

# Review the X_test DataFrame
display(X_test.head())
display(X_test.tail())

,SMA_fast,SMA_slow
timestamp,,
2021-10-31 05:00:00+00:00,4104.749333,3560.7438
2021-11-01 05:00:00+00:00,4128.462667,3580.0200
2021-11-02 05:00:00+00:00,4161.994667,3599.2724
2021-11-03 05:00:00+00:00,4210.354000,3624.5578
2021-11-04 05:00:00+00:00,4258.556000,3648.2676


,SMA_fast,SMA_slow
timestamp,,
2022-10-23 05:00:00+00:00,1302.236667,1404.9070
2022-10-24 05:00:00+00:00,1304.550667,1400.7362
2022-10-25 05:00:00+00:00,1306.006667,1396.1220
2022-10-26 05:00:00+00:00,1319.584667,1392.9224
2022-10-27 05:00:00+00:00,1337.580667,1393.9290


In [670]:
# Scale the features DataFrames

# Create a StandardScaler instance
scaler = StandardScaler()

# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [671]:
# From SVM, instantiate SVC classifier model instance
svm_model = svm.SVC()

# Fit the model to the data using the training data
svm_model = svm_model.fit(X_train_scaled, y_train)
 
# Use the testing data to make the model predictions
svm_pred = svm_model.predict(X_test_scaled)

# Review the model's predicted values
svm_pred[:10]
        

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [672]:
# Use a classification report to evaluate the model using the predictions and testing data
svm_testing_report = classification_report(y_test, svm_pred)

# Print the classification report
print(svm_testing_report)

              precision    recall  f1-score   support

         0.0       0.92      0.50      0.65       260
         1.0       0.41      0.88      0.56       102

    accuracy                           0.61       362
   macro avg       0.66      0.69      0.60       362
weighted avg       0.77      0.61      0.62       362



In [673]:
# Create a new empty predictions DataFrame:

# Create a predictions DataFrame
predictions_df = pd.DataFrame(index=X_test.index)

# Add the SVM model predictions to the DataFrame
predictions_df['Predicted'] = svm_pred

# Add the actual returns to the DataFrame
predictions_df['Actual Returns'] = ETH_signals_df["Actual Returns"]

# Add the strategy returns to the DataFrame
predictions_df['Strategy Returns'] = ETH_signals_df["Actual Returns"]*predictions_df["Predicted"]

# Review the DataFrame
display(predictions_df.head())
display(predictions_df.tail())

,Predicted,Actual Returns,Strategy Returns
timestamp,,,
2021-10-31 05:00:00+00:00,1.0,-0.032032,-0.032032
2021-11-01 05:00:00+00:00,1.0,0.032097,0.032097
2021-11-02 05:00:00+00:00,1.0,0.040322,0.040322
2021-11-03 05:00:00+00:00,1.0,0.008608,0.008608
2021-11-04 05:00:00+00:00,1.0,-0.027580,-0.027580


,Predicted,Actual Returns,Strategy Returns
timestamp,,,
2022-10-23 05:00:00+00:00,1.0,0.028986,0.028986
2022-10-24 05:00:00+00:00,1.0,-0.000964,-0.000964
2022-10-25 05:00:00+00:00,1.0,0.099688,0.099688
2022-10-26 05:00:00+00:00,1.0,0.054753,0.054753
2022-10-27 05:00:00+00:00,1.0,-0.021185,-0.021185


In [674]:
# Plot the actual returns versus the strategy returns
(1 + predictions_df[["Actual Returns", "Strategy Returns"]]).cumprod().hvplot(title="ETH returns with SMA/SVM",width=900,height=400)

:NdOverlay   [Variable]
   :Curve   [timestamp]   (value)